## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [2]:
df = pd.read_csv('redfin_2023-05-01-12-53-54.csv')

## Data Clean

In [3]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [4]:
df = df.dropna(subset=['SOLD DATE'])

In [6]:
# Define list of desired months (excluding current month)
desired_months = ['April']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [7]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

False    962
Name: PRICE, dtype: int64
-------
False    949
True      13
Name: $/SQUARE FEET, dtype: int64
-------
False    962
Name: YEAR BUILT, dtype: int64
-------


In [8]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [8]:
# df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
# df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
# df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
# df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
# df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [9]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
912,PAST SALE,April-14-2023,Condo/Co-op,1450 SE 3rd Ave #407,Dania Beach,FL,33004.0,300.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Dania/1450-SE-3rd-Av...,MARMLS,A11257105,N,Y,26.034462,-80.139805
776,PAST SALE,April-28-2023,Condo/Co-op,6080 NW 44th St #201,Lauderhill,FL,33319.0,16000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/6080-NW-4...,MARMLS,A11323992,N,Y,26.178912,-80.232357
152,PAST SALE,April-25-2023,Condo/Co-op,6300 NW 62 St #205,Tamarac,FL,33319.0,75000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Tamarac/6300-NW-62nd...,Beaches MLS,F10366334,N,Y,26.200585,-80.234977
447,PAST SALE,April-5-2023,Condo/Co-op,4881 NW 22nd St Unit B2,Lauderhill,FL,33313.0,75000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/4881-NW-2...,Beaches MLS,F10356634,N,Y,26.154336,-80.216315
30,PAST SALE,April-12-2023,Condo/Co-op,9301 Lime Bay Blvd #102,Tamarac,FL,33321.0,75000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Tamarac/9301-Lime-Ba...,MARMLS,A11353134,N,Y,26.207841,-80.276231
405,PAST SALE,April-17-2023,Condo/Co-op,1300 NW 43rd Ter #107,Lauderhill,FL,33313.0,77000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/1300-NW-4...,Beaches MLS,RX-10865063,N,Y,26.142565,-80.207641
411,PAST SALE,April-17-2023,Condo/Co-op,1300 NW 43 Ter #104,Lauderhill,FL,33313.0,77000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/1300-NW-4...,Beaches MLS,RX-10865050,N,Y,26.142565,-80.207641
629,PAST SALE,April-24-2023,Condo/Co-op,1501 NW 43rd Ave #203,Lauderhill,FL,33313.0,78000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/1501-NW-4...,Beaches MLS,F10368195,N,Y,26.144276,-80.207203
785,PAST SALE,April-21-2023,Condo/Co-op,9360 Lime Bay Blvd #202,Tamarac,FL,33321.0,80000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Tamarac/9360-Lime-Ba...,Beaches MLS,F10359873,N,Y,26.206294,-80.277054
395,PAST SALE,April-28-2023,Condo/Co-op,2301 NW 41st Ave #303,Lauderhill,FL,33313.0,80000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/2301-NW-4...,MARMLS,A11359324,N,Y,26.155719,-80.204461


In [29]:
print(df_filtered['URL'].iloc[776])

https://www.redfin.com/FL/Deerfield-Beach/427-Grantham-B-33442/unit-427/home/113042473


In [11]:
# Correct the prices, if needed
# df_filtered.at[618,'PRICE']=(295000)
# df_filtered.at[832,'PRICE']=(355000)

In [12]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [27]:
# Corrections, if needed
df_filtered.at[912,'$/SQUARE FEET']=(300000/1020)

In [28]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
776,16000.0,6080 NW 44th St #201,Lauderhill,15.0
629,78000.0,1501 NW 43rd Ave #203,Lauderhill,82.0
278,94370.0,7080 Environ Blvd #324,Lauderhill,86.0
447,75000.0,4881 NW 22nd St Unit B2,Lauderhill,90.0
85,88000.0,2251 NW 48th Ter #114,Lauderhill,95.0
732,115000.0,3360 Spanish Moss Ter #110,Lauderhill,96.0
638,90000.0,4740 NW 21st St #312,Lauderhill,97.0
891,95000.0,4046 NW 19th St #406,Lauderhill,97.0
903,90000.0,4750 NW 22nd Ct #516,Lauderhill,98.0
42,85000.0,3251 Holiday Springs Blvd #203,Margate,98.0


In [14]:
print(df_filtered.URL.iloc[912])

https://www.redfin.com/FL/Dania/1450-SE-3rd-Ave-33004/unit-407/home/42026494


In [16]:
# Corrections, if needed
df_filtered.at[912,'PRICE']=(300000)

In [17]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [17]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [18]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [20]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [21]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [19]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [20]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"April 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [21]:
m.save('index.html')

## Summary Info

In [22]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [23]:
df_filtered.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'PRICE_AS_CURRENCY',
       'YEAR BUILT DISPLAY', 'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [24]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [25]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Four Seasons, 525 N Ft Lauderdale Bch #702 Fort Lauderdale | Price $4,200,000 | $1,878 psf | Year built: 2022
Least Expensive
GREENS OF INVERRARY PHASE, 6080 NW 44th St #201 Lauderhill | Price $16,000 | $15 psf | Year built: 1974
Highest Price Per Square Foot
Four Seasons, 525 N Ft Lauderdale Bch Blvd #1604 Fort Lauderdale | Price $4,000,000 | $1,933 psf | Year built: 2022
Lowest Price Per Square Foot
SHERIDAN EAST CONDO BLDG, 1450 SE 3rd Ave #407 Dania Beach | Price $300,000 | $0 psf | Year built: 1998
Newest
Four Seasons, 525 N Ft Lauderdale Bch #702 Fort Lauderdale | Price $4,200,000 | $1,878 psf | Year built: 2022
Oldest
HOLIDAY ISLE YACHT CLUB C, 741 Bayshore Dr Unit 6S Fort Lauderdale | Price $355,000 | $632 psf | Year built: 1951


## Time on Market Calculator

In [29]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [36]:
from datetime import datetime, timedelta

date1 = datetime(2022, 12, 15) ## List (Earlier) date
date2 = datetime(2023, 3, 30) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

105


## Map URL Snagger

In [31]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [32]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_condo_sales_month_ending_march_2023


## Get Summary Data

In [33]:
print('SALES INFO')
print(f'Number of sales: {len(df_Current)}')
print('--------')
print(f'Total sale price: ${df_Current["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_Current["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_Current["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_Current["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_Current["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_Current["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_Current["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_Current["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_Current["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_Current["YEAR BUILT"].mean()}')

SALES INFO


NameError: name 'df_Current' is not defined